In [ ]:
import math
import numpy as np
import skimage
from skimage import color, io

import scipy

from platform import python_version
print(python_version())

from pylab import *

import skimage
from skimage import color, io
import numpy as np
import scipy
from numba import njit

import scipy.ndimage

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll=function(lines){
    return false;
}

## Prvi zadatak

Prelazimo u LAB iz RGB sistem zato što iz a i b komponente možemo jednoynačno odrediti prisustvo crvene i plave boje, takođe očekujemo da će biti manja osetljivost na osvetljaj, zato što informaciju o osvetljaju sadrži L maska.

In [ ]:




img = io.imread(f".\dices\dices10.jpg")
img = skimage.img_as_float(img)
img =  img - skimage.filters.gaussian(img, sigma=3.5, mode='nearest', truncate=6)*0.25

imgL = color.rgb2lab(img)



fig, [axes, ax1, ax2] = plt.subplots(nrows=3, ncols=1, figsize=(16,16), dpi=120)

im = axes.imshow(imgL[:, :, 0]); axes.set_axis_off(); axes.set_title("L maska")
fig.colorbar(im, ax=axes, fraction=0.03, pad=0.04)

im = ax1.imshow(imgL[:, :, 1]); ax1.set_axis_off(); ax1.set_title("a maska")
fig.colorbar(im, ax=ax1, fraction=0.03, pad=0.04)

im = ax2.imshow(imgL[:, :, 2]); ax2.set_axis_off(); ax2.set_title("b maska")
fig.colorbar(im, ax=ax2, fraction=0.03, pad=0.04)

Sledeći korak je segmentacija, primenićemo tehniku postavljanja praga, u ovom slučaju znamo apriori da radimo sa crvenim i plavim tačkicama, pa možemo usvojiti nivo praga statički, moguće je odrediti ga i dinamički, ali u ovom slučaju za time nema potrebe.

In [ ]:
def image_threshold(image, threshold=1):
    
    # prikaz slike, histograma i pozicija pragova
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 5), dpi=80)
    axes[0].imshow(image, cmap='gray'), axes[0].set_title('Ulazna slika'), axes[0].axis('off')
    
    axes[1].imshow(image>threshold, cmap='gray'), axes[1].set_title('Binarizovana slika'), axes[1].axis('off')
    tight_layout()
    plt.show()
    
    return



image = imgL[:, :, 1]
image_threshold(image, 29)

image = imgL[:, :, 2]
image_threshold(image, -27)



Kao što vidimo segmentacija je uspešno izvršena, nema potrebe za dodatnim poboljšanjem reyultata segmentacije zato što su svi regioni od interesa jasno vidljivi. Sledeći korak je da izbrojimo crvene i plave regione, to ćemo postići pomoću funkcije iz morphologz paketa.


Brojimo samo regione čija je površina veća od 60.

In [ ]:


image = imgL[:, :, 1]

th = 29
image_threshold(image, th)
    
th = 29
image_for_labeling = uint8(scipy.ndimage.median_filter(image>th, size=(3,3), mode='reflect'))

labeled_image = skimage.measure.label(image_for_labeling, background=0)
regions = skimage.measure.regionprops(labeled_image)

# Prolazak kroz sve objekte i ispis parametara Area
suma = 0
for i, region in zip(range(len(regions)), regions):
    if(region['Area'] > 60):
        suma+=1
    
print("Suma crvenih : " + str(suma))

image = imgL[:, :, 2]
th = np.quantile(image, 1 - 0.998)
th = -27
image_threshold(image, th)
    
image_for_labeling = uint8(scipy.ndimage.median_filter(image>th, size=(3,3), mode='reflect'))

labeled_image = skimage.measure.label(image_for_labeling, background=1)
regions = skimage.measure.regionprops(labeled_image)

suma = 0
for i, region in zip(range(len(regions)), regions):
    if(region['Area'] > 60):
        suma+=1
    
print("Suma plavih : " + str(suma))


Uspešno smo izvršili prebrojavanje crvenih i plavih tačkica. Sledi provera postupka nad celim skupom slika.

In [ ]:


for i in range(1,12):
    img = io.imread(f".\dices\dices{i}.jpg")
    img = skimage.img_as_float(img)
    img =  img - skimage.filters.gaussian(img, sigma=3.5, mode='nearest', truncate=6)*0.25

    imgL = color.rgb2lab(img)
    




    image = imgL[:, :, 1]
    th = np.quantile(image, 0.9991)
    th = 29
    image_threshold(image, th)
    
    image_for_labeling = uint8(scipy.ndimage.median_filter(image>th, size=(3,3), mode='reflect'))

    labeled_image = skimage.measure.label(image_for_labeling, background=0)
    regions = skimage.measure.regionprops(labeled_image)

    # Prolazak kroz sve objekte i ispis parametara Area
    suma = 0
    for i, region in zip(range(len(regions)), regions):
        if(region['Area'] > 60):
            suma+=1
    
    print("Suma crvenih : " + str(suma))

    image = imgL[:, :, 2]
    th = np.quantile(image, 1 - 0.998)
    th = -27
    image_threshold(image, th)
    
    image_for_labeling = uint8(scipy.ndimage.median_filter(image>th, size=(3,3), mode='reflect'))

    labeled_image = skimage.measure.label(image_for_labeling, background=1)
    regions = skimage.measure.regionprops(labeled_image)

    suma = 0
    for i, region in zip(range(len(regions)), regions):
            if(region['Area'] > 60):
                suma+=1
    
    print("Suma plavih : " + str(suma))
    suma = 0 

    



In [ ]:

def extract_dice_score(img):
    
    imgL = color.rgb2lab(img - skimage.filters.gaussian(img, sigma=3.5, mode='nearest', truncate=6)*0.25)
    #Prebacujemo sliku u Lab sistem, pre prelaska uklanjamo NF deo slike koji potice od ne unformnog osvetljaja
    
    
    th = 29
    image = imgL[:, :, 1]
    #Znamo da je boja na kockama crvena, pa vrdnost a komponente treba da bude visoka, kao prag smo uvojili 29
    
    image_for_labeling = uint8(scipy.ndimage.median_filter(image>th, size=(3,3), mode='reflect'))

    labeled_image = skimage.measure.label(image_for_labeling, background=0)
    regions = skimage.measure.regionprops(labeled_image)

    # Brojimo samo regione cija je povrsina veca od 60 kako bismo izbegli brojanje brojeva sa bokova kockica
    red_score = 0
    for i, region in zip(range(len(regions)), regions):
        if(region['Area'] > 60):
            red_score+=1
    

    image = imgL[:, :, 2]
    th = -27
     #Znamo da je boja na kockama plava, pa vrdnost b komponente treba jako niska, kao prag smo uvojili -27
    
    image_for_labeling = uint8(scipy.ndimage.median_filter(image>th, size=(3,3), mode='reflect'))

    labeled_image = skimage.measure.label(image_for_labeling, background=1)
    regions = skimage.measure.regionprops(labeled_image)

    blue_score = 0
    for i, region in zip(range(len(regions)), regions):
            if(region['Area'] > 60):
                blue_score+=1
    
    
    
    return [blue_score, red_score]

Vizuelnom inspekcijom možemo da utvrdimo da algoritam daje dobar rezultat za sve ulazne slike.

In [ ]:


for i in range(1,13):
    img = io.imread(f".\dices\dices{i}.jpg")
    img = skimage.img_as_float(img)
    scores = extract_dice_score(img)
    print(f"Image dices{i}.jpg \nBlue Score: {scores[0]} \nRed Score: {scores[1]}\n---------" )

    imgL = color.rgb2lab(img)
    

## Drugi zadatak

Parametri u narednim primerima su podešavani na sledeći način

1) sigma se povećava ako je šum previše izražen, to se najbolje vidi iz kvantizovanog ugla gradijenta, ako je šum previše jak raspodela uglova će biti uniformna, a ako preteramo sa filtriranjem možemo da uočimo formiranje blokova na slici sa uglovima

2) HT se podesi tako da bude iznad nivoa šuma, tj. trudimo se da obuhvatimo samo jake ivice, to se podešava vizuelnom inspekcijom, ako je nivo previše nizak to se vidi u tome što se ivice pojavljuju i tamo gde ih nema

3) LT se podesi tako da obuhvati sve ivice od interesa, značajno utiče na krajni izgled, ako je previše visok nećemo detektivati ivice uspešno, a ako je previše nizak propustićemo previše lažnih ivica

In [ ]:
def gaussian_filter(std, radius):
    N = 2*radius + 1
    
    y = np.arange(0,N) - (N-1)/2
    x = np.arange(0,N) - (N-1)/2

    X, Y = meshgrid(x, y)
    
    D = np.square(X) + np.square(Y)
    
    gaussian_mask = exp(-D/(2*np.square(std)))
    
    gaussian_mask = gaussian_mask/(np.sum(gaussian_mask))
    
    return gaussian_mask
    

def gradient_x(img, K):
    hx = np.array([[-1, 0, 1]]).T
    h = np.array([[1, K, 1]])

    
    grad_x = hx.dot(h)
    
    
    return scipy.ndimage.convolve(img, grad_x, mode='nearest')

def gradient_y(img, K):
    hy = np.array([[-1, 0, 1]])
    h = np.array([[1, K, 1]]).T

    grad_y = h.dot(hy)
    
    
    return scipy.ndimage.convolve(img, grad_y,mode='nearest')
def gradient(img, K):
    g_x = gradient_x(img, K)
    g_y = gradient_y(img, K)
    
    angle = np.arctan2(g_y, g_x)
    
    return (g_y, g_x, angle)
    
def quantize(angle):
    
    angle[np.logical_or(np.logical_or(np.logical_and(angle<=22.5,angle>-22.5), angle>157.5),angle<-157.5)] = 0
    angle[np.logical_or(np.logical_and(angle<=112.5, angle>67.5), np.logical_and(angle<=-67.5, angle>-112.5))] = 90
    angle[np.logical_or(np.logical_and(angle<=157.5, angle>112.5), np.logical_and(angle<=-22.5, angle>-67.5))] = 135
    angle[np.logical_or(np.logical_and(angle<=67.5,angle>22.5),np.logical_and(angle<=-112.5, angle>-157.5))] = 45
    angle[angle==45]=-45
    angle[angle==135] = 45
    
    return angle



def remove_non_extremum(magnitude, angle, radius):
    Nx = angle.shape[0]
    Ny = angle.shape[1]
    mag = np.zeros((Nx, Ny))
    
    for x in range(radius, Nx - radius):
        for y in range(radius, Ny - radius):
        
            idx = (x, y)
            alpha = angle[idx]
            ## i-1,j-1  i-1,j  i-1,j+1 
            ## i,j-1    i,j    i,j+1
            ## i+1,j-1  i+1,j  i+i,j+1
            mag[idx] = magnitude[idx]
            if(alpha == -45):
                a = (x-1, y-1)
                c = (x+1, y+1)
            if(alpha == 0):
                a = (x-1, y)
                c = (x+1, y)
            if(alpha == 45):
                a = (x-1, y+1)
                c = (x+1, y-1)
            if(alpha == 90):
                a = (x, y+1)
                c = (x, y-1)

                
            if(magnitude[idx] != max(magnitude[a], magnitude[idx], magnitude[c])):
                mag[idx] = 0
            
    return mag
    
def connect_edges(strong, weak, radius):
    
    connected = np.array(strong)
    temp = np.array(weak)
    
    Ny = strong.shape[0]
    Nx = strong.shape[1]
    
    process = True
   
    while process:
        process = False
        for i in range(radius, Nx - radius):
            for j in range(radius, Ny - radius):
                idx = (j, i)
                ## a b c 
                ## d e f
                ## g h k
                a = (j-1, i-1)
                b = (j, i-1)
                c = (j+1, i-1)
                d = (j-1, i)
                e = (j, i)
                f = (j+1, i)
                g = (j-1, i+1)
                h = (j, i+1)
                k = (j+1, i+1)
            
                positions = [a, b, c, d, e, f, g, h, k]
            
                if(temp[idx]!=0):
                    for pos in positions:
                        if(connected[pos]!=0):
                            temp[idx]=0
                            process = True
                            connected[idx]=1
                            break
    return connected

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def canny_edge_detection(img, std, Th, Tl, K = 2):
    radius = math.ceil(6*std)
    N_extra = radius
    Ny = img.shape[0]
    Nx = img.shape[1]
    
    std = np.abs(std)
    Th = np.abs(Th)
    Tl = np.abs(Tl)
   #Sobel gradient operator with K = 2
    
    gaussian_mask = gaussian_filter(std, radius)
    
    
    if(len(img.shape) == 3):
        layer = np.pad(np.array(rgb2gray(img)), radius, mode='edge')
    else:
        layer = np.pad(np.array(img), radius, mode='edge')
    

    #layer = scipy.ndimage.correlate(layer, gaussian_mask)
    layer =  skimage.filters.gaussian(layer, sigma=std, mode='nearest', truncate=6)
        
    g_y_o, g_x_o, angle = gradient(layer, K)
    #mag = np.array(magnitude[N_extra : N_extra + Ny, N_extra :N_extra + Nx])
        
    #Conversion to degrees
    angle = np.rad2deg(angle)
        
    angle = quantize(angle)
        
    
    g_y = remove_non_extremum(np.abs(g_y_o), angle, radius)
    g_x = remove_non_extremum(np.abs(g_x_o), angle, radius)
    
    
    figure, ax=subplots(nrows=2,ncols=2, figsize=(20,16), dpi=80)

    im = ax[0,0].imshow(np.abs(g_y_o), cmap='gray')
    ax[0,0].set_title('Magnituda gradijenta y'), ax[0,0].axis('off')
    im0 = ax[0,1].imshow(np.abs(g_x_o), cmap='gray')
    ax[0,1].set_title('Magnituda gradijenta x'), ax[0,1].axis('off')
    im1 = ax[1,0].imshow(g_y, cmap='gray')
    ax[1,0].set_title('Magnituda gradijenta y potisnuto'), ax[1,0].axis('off')
    im2 = ax[1,1].imshow(g_x, cmap='gray')
    ax[1,1].set_title('Magnituda gradijenta x potisnuto'), ax[1,1].axis('off')
    figure.colorbar(im0, ax=ax[0,0], fraction=0.03, pad=0.04)
    figure.colorbar(im0, ax=ax[0,1], fraction=0.03, pad=0.04)
    figure.colorbar(im1, ax=ax[1,0], fraction=0.03, pad=0.04)
    figure.colorbar(im2, ax=ax[1,1], fraction=0.03, pad=0.04)
    matplotlib.pyplot.tight_layout()
    plt.show()
    
    magnitude = np.sqrt(np.square(g_x) + np.square(g_y))
        
    strong_edge = np.array(magnitude)
    weak_edge = np.array(magnitude)
        
    strong_edge[magnitude >= Th] = 1
    strong_edge[magnitude < Th] = 0
        
    weak_edge[magnitude >= Tl] = 1
    weak_edge[magnitude < Tl] = 0
    weak_edge[magnitude >= Th] = 0
        
    canny = connect_edges(strong_edge, weak_edge, radius)
        
    #Padding removal 
    canny = canny[N_extra : N_extra + Ny, N_extra :N_extra + Nx]
    magnitude = magnitude[N_extra : N_extra + Ny, N_extra :N_extra + Nx]
    angle = angle[N_extra : N_extra + Ny, N_extra :N_extra + Nx]
    strong_edge = strong_edge[N_extra : N_extra + Ny, N_extra :N_extra + Nx]
    weak_edge = weak_edge[N_extra : N_extra + Ny, N_extra :N_extra + Nx]
    
    
    # prikaz slike, magnitude i ugla gradijenta
    figure, ax=subplots(nrows=2,ncols=2, figsize=(20,16), dpi=80)

    im = ax[0,0].imshow(magnitude, cmap='gray')
    ax[0,0].set_title('Magnituda gradijenta potisnuto'), ax[0,0].axis('off')
    im0 = ax[0,1].imshow(angle, cmap='jet')
    ax[0,1].set_title('Ugao gradijenta arctan2'), ax[0,1].axis('off')
    im1 = ax[1,0].imshow(strong_edge, cmap='gray')
    ax[1,0].set_title('Jake ivice'), ax[1,0].axis('off')
    im2 = ax[1,1].imshow(weak_edge, cmap='gray')
    ax[1,1].set_title('Slabe ivice'), ax[1,1].axis('off')
    figure.colorbar(im0, ax=ax[0,0], fraction=0.03, pad=0.04)
    figure.colorbar(im0, ax=ax[0,1], fraction=0.03, pad=0.04)
    figure.colorbar(im1, ax=ax[1,0], fraction=0.03, pad=0.04)
    figure.colorbar(im2, ax=ax[1,1], fraction=0.03, pad=0.04)
    matplotlib.pyplot.tight_layout()
    plt.show()
    

    return canny

In [ ]:
img = io.imread('lena.tif')
img = skimage.img_as_float(img)


input_title = 'ulazna slika'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(img, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

Krajnje podešavanje parametara.

Sigma : slika nije zašumljena, pa nije neophodno intezivno filtriranje. Usvajamo sigma = 0.8, to smo odredili na osnovu izgleda ugla gradijenta, vidimo da su uglovi bitnih ivica jasno definisani.

HT : podešavamo tako da obuhvatimo oblik lica, ivicu šešira i ivice u pozadini. Usvajamo HT = 0.35

LT : podešavamo tako da konačne ivice nemaju prekide u sebi. Usvajamo LT = 0.25

In [ ]:


def primena_canny(image, sigma=0.8, lt=0.25, ht=0.35):
    canny =  canny_edge_detection(img, sigma, ht, lt, 2)

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

    axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

    plt.show()
    return


#interact(primena_canny, image = fixed(image), sigma = (0,5,0.1), lt=(0,2,0.02), ht=(0,2,0.05))
canny =  canny_edge_detection(img , 0.8, 0.35, 0.25, 2)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title) 

U ovom primeru smo značajno povećali intezitet filtriranja. Na osnovu ugla gradijenta vidimo da smo donekle preterali, na osnovu reyultata algoritma vidimo da sad uspešno detektujemo samo izražene ivice, što je teoretski i oćekivano zato što NF filtar uništava ivice.

In [ ]:
def primena_canny(image, sigma=1.8, lt=0.15, ht=0.28):
    canny =  canny_edge_detection(img, sigma, ht, lt, 2)

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

    axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

    plt.show()
    return


#interact(primena_canny, image = fixed(image), sigma = (0,5,0.1), lt=(0,2,0.02), ht=(0,2,0.05))

canny =  canny_edge_detection(img , 1.8, 0.28, 0.15, 2)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title) 

U ovom primeru smo značajno smanjili intezitet filtriranja. To možemo ustanoviti na osnovu ugla gradijenta zato što su uglovi jakih ivica lošije određeni, na osnovu rezultat algoritma vidimo da je šum izraženiji, pa je teže uspešno detektovati ivice, to se najbolje vidi u predelu usana.

In [ ]:
def primena_canny(image, sigma=0.1, lt=0.35, ht=0.55):
    canny =  canny_edge_detection(img, sigma, ht, lt, 2)

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

    axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

    plt.show()
    return


#interact(primena_canny, image = fixed(image), sigma = (0,5,0.1), lt=(0,2,0.02), ht=(0,2,0.05))

canny =  canny_edge_detection(img , 0.1, 0.55, 0.35, 2)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title) 

In [ ]:
img = io.imread('house.tif')
img = skimage.img_as_float(img)


input_title = 'ulazna slika'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(img, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

Krajnje podešavanje parametara.

Sigma : slika nije zašumljena, pa nije neophodno intezivno filtriranje. Usvajamo sigma = 0.1, to smo odredili na osnovu izgleda ugla gradijenta, vidimo da su uglovi bitnih ivica jasno definisani.

HT : podešavamo tako da oblik kuće, cigle nisu dovoljno izražene da bi ih detektovali kao ivice. Usvajamo HT = 0.65

LT : podešavamo tako da konačne ivice nemaju prekide u sebi. Usvajamo LT = 0.3

In [ ]:

def primena_canny(image, sigma=0.1, lt=0.30, ht=0.65):
    canny =  canny_edge_detection(img, sigma, ht, lt, 2)

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

    axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

    plt.show()
    return


#interact(primena_canny, image = fixed(image), sigma = (0,5,0.1), lt=(0,2,0.02), ht=(0,2,0.05))

canny =  canny_edge_detection(img , 0.1, 0.65, 0.3, 2)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

In [ ]:
img = io.imread('van.tif')
img = skimage.img_as_float(img)


input_title = 'ulazna slika'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(img, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

Krajnje podešavanje parametara.

Sigma : slika je zašumljena, pa neophodno intezivno filtriranje. Usvajamo sigma = 1, to smo odredili na osnovu izgleda ugla gradijenta, vidimo da su uglovi bitnih ivica jasno definisani.

HT : podešavamo tako da oblik kombija, prozora i tablice. Usvajamo HT = 0.35

LT : podešavamo tako da konačne ivice nemaju prekide u sebi. Usvajamo LT = 0.2

In [ ]:
def primena_canny(image, sigma=1, lt=0.2, ht=0.35):
    canny =  canny_edge_detection(img, sigma, ht, lt, 2)

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

    axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

    plt.show()
    return


#interact(primena_canny, image = fixed(image), sigma = (0,5,0.1), lt=(0,2,0.02), ht=(0,2,0.05))
canny =  canny_edge_detection(img , 1, 0.35, 0.20, 2)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

In [ ]:
img = io.imread('camerman.tif')
img = skimage.img_as_float(img)


input_title = 'ulazna slika'

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(img, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

Krajnje podešavanje parametara.

Sigma : slika je jako zašumljena, pa neophodno intezivno filtriranje. Usvajamo sigma = 1.5, to smo odredili na osnovu izgleda ugla gradijenta, vidimo da su uglovi bitnih ivica jasno definisani.

HT : podešavamo tako da oblik čoveka i kamere sa stativom. Usvajamo HT = 0.72

LT : podešavamo tako da konačne ivice nemaju prekide u sebi. Usvajamo LT = 0.32

In [ ]:
def primena_canny(image, sigma=1.5, lt=0.32, ht=0.72):
    canny =  canny_edge_detection(img, sigma, ht, lt, 2)

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

    axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title)

    plt.show()
    return


#interact(primena_canny, image = fixed(image), sigma = (0,5,0.1), lt=(0,2,0.02), ht=(0,2,0.05))

canny =  canny_edge_detection(img , 1.5, 0.72, 0.32, 2)

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,16), dpi=120)

axes.imshow(canny, cmap = 'gray'); axes.set_axis_off(); axes.set_title(input_title) 